In [1]:
%matplotlib inline

import time
import numpy as np
import progressbar_renamed as progressbar
import copy
import networkx as nx
from operator import itemgetter


import import_ipynb
import QAOA_for_QLS as qaoa
import Side_Calc_QLS as sc

importing Jupyter notebook from QAOA_for_QLS.ipynb
importing Jupyter notebook from Side_Calc_QLS.ipynb


In [2]:
def top_gains_populate_subset(G, subset_size, gains):
    if G.number_of_nodes() <= subset_size:
        return list(G.nodes())
    return [
        x[0] for x in sorted(gains.items(), key=itemgetter(1))[-subset_size:]
    ]

In [3]:
def iteration_step(G, B, curr_solution, subset):
    
    subset2global = dict((x, subset[x]) for x in range(0, len(subset)))
    global2subset = dict((subset[x], x) for x in range(0, len(subset)))
    
    C = [0.0] * len(subset)
    for i in subset:
        for j in set(G.nodes()) - set(subset):
            C[global2subset[i]] += 2 * B[i, j] * curr_solution[j]
    indices = np.array(subset)  # rows and columns of B to keep for subset
    
    result = qaoa.qaoa_basic(len(subset), B[np.ix_(indices, indices)],C)
    optimized_subset=result.x
    
        
    if 0 in optimized_subset:
        optimized_subset = [
            -1 if x == 0 else 1 if x == 1 else 'Error' for x in optimized_subset
        ]
        
    for i in range(0, len(optimized_subset)):
        curr_solution[subset2global[i]] = optimized_subset[i]
    return curr_solution



def QLS(G,size_of_iteration=12,stopping_criteria=3):
    B = nx.modularity_matrix(G, nodelist=sorted(G.nodes()), weight='weight')
    
    # random initial guess
    curr_solution = [
        1 - 2 * x
        for x in list(np.random.randint(2, size=(G.number_of_nodes())))
    ]
    curr_modularity = sc.compute_modularity(G, B, curr_solution)
    
    all_time_best_solution = curr_solution
    all_time_best_modularity = curr_modularity

    visited = set()
    it = 0
    it_stuck = 0
    all_modularities = []
    
    while set(G.nodes()) - visited:
        it += 1
        if it_stuck > stopping_criteria: 
            break
        gains_list = []
        for v in progressbar.progressbar(G.nodes()):
            gains_list.append(sc.compute_gain(G, B, curr_solution, v, True))
        gains = {v: gain for gain, v in gains_list} 
        
        subset = top_gains_populate_subset(G, size_of_iteration, gains)
        
        cand_solution = iteration_step(G, B, copy.deepcopy(curr_solution), list(subset))
        cand_modularity = sc.compute_modularity(G, B, cand_solution)
        
        print('it', it, 'cand_modularity', cand_modularity, 'curr_best',
              all_time_best_modularity)
        all_modularities.append({
            'it': it,
            'cand_modularity': cand_modularity,
            'curr_best': all_time_best_modularity
        })
        if cand_modularity > curr_modularity:
            curr_solution = cand_solution
            curr_modularity = cand_modularity
            it_stuck = 0
        else:
            it_stuck += 1
        if curr_modularity > all_time_best_modularity:
            all_time_best_solution = curr_solution
            all_time_best_modularity = curr_modularity
        
        #break Bedingung?, vgl.: single_level_refinement
    return (all_time_best_modularity, all_time_best_solution, it,
            all_modularities)
        

In [4]:
G=nx.barbell_graph(4,0)

In [5]:
#qaoa.qaoa_basic(8,G)

In [7]:
G2=nx.barbell_graph(20,0)
#QLS(G2)

In [8]:
#Probe Implementierung qaoa_basic
result = qaoa.qaoa_basic(8, nx.modularity_matrix(G))
optimized_subset=result.x

print(optimized_subset)

[1. 1. 1. 1. 0. 0. 0. 0.]
